In [6]:
import pandas as pd
from urllib.request import urlopen
import json
from datetime import datetime

In [46]:
# Connect to MongoDB
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27")
mongodb = client.chessDB
gameCollection = mongodb.gameCollection
playerCollection = mongodb.playerCollection

In [47]:
#calculate player rating based on formula
def get_rating(data_json):
    chess_rapid_rating =chess_bullet_rating=chess_blitz_rating=0
    rd_blitz=rd_bullet=rd_rapid=0
  
    try:
        rd_rapid= data_json["chess_rapid"]["last"]["rd"]
        weight_rapid = (1/rd_rapid**2)
        chess_rapid_rating= data_json["chess_rapid"]["last"]["rating"]
    except:
        chess_rapid_rating=0
  
    try:
        rd_bullet= data_json["chess_bullet"]["last"]["rd"]
        weight_bullet = (1/rd_bullet**2)
        chess_bullet_rating = data_json["chess_bullet"]["last"]["rating"]
    except:
        chess_bullet_rating=0
    
    try:
        rd_blitz= data_json["chess_blitz"]["last"]["rd"]
        weight_blitz = (1/rd_blitz**2)
        chess_blitz_rating= data_json["chess_blitz"]["last"]["rating"]
    except:
        chess_blitz_rating=0

    return int((chess_rapid_rating*weight_rapid + chess_bullet_rating*weight_bullet + chess_blitz_rating*weight_blitz)/(weight_rapid+weight_bullet+weight_blitz))

In [48]:
# get wins losses and draws for a data_json
def get_wld (data_json):
  
  wins1=losses1=draws1=wins2=losses2=draws2=wins3=losses3=draws3=0
  # try:    
  #   wins1,losses1,draws1= data_json["chess_rapid"]["record"].values()
  #   wins2,losses2,draws2= data_json["chess_bullet"]["record"].values()
  #   wins3,losses3,draws3= data_json["chess_blitz"]["record"].values() 
  # except:
  #   pass
  try:
    rapid_record = data_json.get("chess_rapid", {}).get("record", {})
    wins1,losses1,draws1 = rapid_record["win"] or 0, rapid_record["loss"] or 0, rapid_record["draw"] or 0

  except KeyError as e:
    pass

  try:
    bullet_record = data_json.get("chess_bullet", {}).get("record", {})
    wins2, losses2, draws2 = bullet_record.get("win", 0), bullet_record.get("loss", 0), bullet_record.get("draw", 0)
  except KeyError as e:
    pass

  try:
    blitz_record = data_json.get("chess_blitz", {}).get("record", {})
    wins3, losses3, draws3 = blitz_record.get("win", 0), blitz_record.get("loss", 0), blitz_record.get("draw", 0)
  except KeyError as e:
    pass
  # print(f"{wins3},{losses3},{draws3}")

  wins = wins1+wins2+wins3
  losses= losses1+losses2+losses3
  draws = draws1+draws2+draws3

  return wins,losses,draws

In [49]:
def getDate(data_json):
    
    try:
        dates = [data_json["chess_blitz"]["last"]["date"], data_json["chess_bullet"]["last"]["date"], data_json["chess_rapid"]["last"]["date"]]
        unixDate = max(dates)
        date = datetime.datetime.fromtimestamp(unixDate).strftime('%Y.%m.%d')
    except:
        date = datetime.today().strftime('%Y.%m.%d')

    return date

In [50]:
titles = ["GM", "WGM", "IM", "WIM", "FM", "WFM", "NM", "WNM", "CM", "WCM"]
names = []
playerDocuments = playerCollection.find()

allNames = []
for doc in playerDocuments:
    try:
        allNames.append(doc['name'])
    except:
        print(doc)

for title in titles:
  url = "https://api.chess.com/pub/titled/{}".format(title)
  response = urlopen(url)
  data_json = json.loads(response.read())
  names.extend(data_json["players"])

players={}
chess_rapid_rating = chess_bullet_rating = chess_blitz_rating = 0
rd_blitz = rd_bullet = rd_rapid = 0
weight_rapid = weight_bullet = weight_blitz = 0

for id in names:
    url_name = "https://api.chess.com/pub/player/{}/stats".format(id)
    try:
        response = urlopen(url_name)
        data_json = json.loads(response.read())
        rating = get_rating(data_json)
    except:
        continue
    
    wins, losses, draws = get_wld(data_json)
    date = getDate(data_json)

    url_name = "https://api.chess.com/pub/player/{}".format(id)
    response = urlopen(url_name)
    data_json = json.loads(response.read())
    
    try:
        name = data_json['name']
    except:
        continue
    
    if name in allNames:
    # update wins losses or draws
        myquery = {"name": name}
        myPlayer = playerCollection.find_one(myquery)
        if myPlayer is None:
            continue
    
        newvalues = { "$set": { "wins": wins, "losses": losses, "draws": draws } }
        playerCollection.update_one(myquery, newvalues)

        #update date
        newDate = myPlayer['dates']
        newDate.append(date)
        newvalues = { "$set": { "dates": newDate } }
        playerCollection.update_one(myquery, newvalues)

        #update rating
        newRating = myPlayer['ratings']
        newRating.append(rating)
        newvalues = { "$set": { "ratings": newRating} }
        playerCollection.update_one(myquery, newvalues)

    else:
        playerData = {'name': name,
                    'wins': wins,
                    'losses': losses,
                    'draws': draws,
                    'dates':[date],
                    'ratings':[rating]
                    }
        playerCollection.insert_one(playerData)
        allNames.append(name)

In [51]:
YY = [2022, 2023]
MM = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
final_list = []

titles = ["GM", "WGM", "IM", "WIM", "FM", "WFM", "NM", "WNM", "CM", "WCM"]
names = []

for title in titles:
  url = "https://api.chess.com/pub/titled/{}".format(title)
  response = urlopen(url)
  data_json = json.loads(response.read())
  names.extend(data_json["players"])

current_month = datetime.datetime.now().strftime("%m")
current_year = datetime.datetime.now().year


for name in names:
  for y in YY:
    for m in MM:
      try:
        url = "https://api.chess.com/pub/player/{}/games/{}/{}".format(name, current_year, current_month)
        response = urlopen(url)
        data_json = json.loads(response.read())

        for game in data_json.get("games"):
          match = game.get("pgn").replace("[", "").replace("]", "").split("\n")
          match.remove("")
          match.remove("")
          game_dict = {}
          header_dict = {}
          for i in range(len(match) - 1):
            key = match[i].split('"')[0].replace(" ", "")
            val = match[i].split('"')[1]
            header_dict[key] = val
          mov = match[len(match) - 1].split(" ")
          arr = []
          moves_arr = []
          str = ''
          for i in range(0, len(mov)-1):
            if mov[i].find('.') != -1 or mov[i].find('{') != -1 or mov[i].find(':') != -1 or mov[i].find('}') != -1:
              str = ''
            else:
              str = str+mov[i]
            if len(str) != 0:
              arr.append(str)
          for j in range(len(arr)-1):
            if j % 2 == 0:
              moves_arr.append(arr[j]+arr[j+1])
          game_dict['header'] = header_dict    
          game_dict['moves'] = moves_arr
          # print(game_dict)
          final_list.append(game_dict)
      except:
        continue

keys_to_remove = ['Site', 'Round', 'CurrentPosition', 'Timezone', 'ECOUrl', 'UTCDate', 'UTCTime', 'TimeControl', 'Termination', 'StartTime', 'EndDate', 'EndTime', 'Link']

for i in range(len(final_list)):  
  for key in keys_to_remove:
      final_list[i]['header'].pop(key, None)

gameCollection.insert_many(final_list)